In [ ]:
import numpy as np 
from collections import Counter
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import confusion_matrix
from datetime import datetime
import math
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# Decision stump used as greedy classifier
class DecisionStump:
    def __init__(self):
        self.polarity = 1
        self.feature_idx = None
        self.threshold = None
        self.alpha = None

    def predict(self, X):
        n_samples = X.shape[0]
        X_column = X.iloc[:, self.feature_idx]
        predictions = np.ones(n_samples)
        if self.polarity == 1:
            predictions[X_column < self.threshold] = -1
        else:
            predictions[X_column >= self.threshold] = -1

        return predictions


class Adaboost:
    def __init__(self, n_clf=5):
        self.n_clf = n_clf
        self.clfs = []

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialize weights to 1/N
        w = np.full(n_samples, (1 / n_samples))

        self.clfs = []

        # Iterate through classifiers
        for _ in range(self.n_clf):
            clf = DecisionStump()
            min_error = float("inf")

            # greedy search to find best threshold and feature
            for feature_i in range(n_features):
                X_column = X.iloc[:, feature_i]
                thresholds = np.unique(X_column)

                for threshold in thresholds:
                    # predict with polarity 1
                    p = 1
                    predictions = np.ones(n_samples)
                    predictions[X_column < threshold] = -1

                    # Error = sum of weights of misclassified samples
                    misclassified = w[y != predictions]
                    error = sum(misclassified)

                    if error > 0.5:
                        error = 1 - error
                        p = -1

                    # store the best configuration
                    if error < min_error:
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_idx = feature_i
                        min_error = error

            # calculate alpha
            EPS = 1e-10
            clf.alpha = 0.6 * np.log((1.0 - min_error + EPS) / (min_error + EPS))

            # calculate predictions and update weights
            predictions = clf.predict(X)

            w *= np.exp(-clf.alpha * y * predictions)
            # Normalize to one
            w /= np.sum(w)

            # Save classifier
            self.clfs.append(clf)

    def predict(self, X):
        clf_preds = [clf.alpha * clf.predict(X) for clf in self.clfs]
        y_pred_value = np.sum(clf_preds, axis=0)
        y_pred = np.sign(y_pred_value)

        return y_pred, y_pred_value

In [ ]:
# TEST DECISIONSTUMP AND ADABOOST FUNCTION
start_time = datetime.now()
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

#import dataset
#data = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\covtype.csv", header = 0, sep=",")
data = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct1.csv")
data2 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct2.csv")
data3 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct3.csv")
data4 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct4.csv")
data5 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct5.csv")
data6 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct6.csv")
data7 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct7.csv")
data = data.append(data2)
data = data.append(data3)
data = data.append(data4)
data = data.append(data5)
data = data.append(data6)
data = data.append(data7)
data_cont = data.loc[:, 'Elevation':'Horizontal_Distance_To_Fire_Points'] # continuous variable
data = data.reset_index(drop = True)
#bunching cont variable
for cont_col in data_cont.columns:
        #check every feat is >0
        if data[cont_col].min() > 0:
            #log transformation
            data[cont_col] = np.log(data[cont_col])
        data[cont_col] = pd.qcut(data[cont_col], q = 15, labels = False) #from continuous to class, range(0, 10)
        
#define X and Y
X_init = data.iloc[:, :-1]
y_init = data.iloc[:, data.shape[1]-1]



#split in train and test (for each binary classifier we have the same train and test)
X_train_init, X_test_init, y_train_init, y_test_init = train_test_split(X_init, y_init, test_size = 0.20, random_state = 1234) 

#Hyperparamethers tuning (no external cross validation)
T_round = [2, 5, 10, 20, 50, 100, 150]

train = pd.concat([X_train_init, y_train_init], axis = 1)
shuffled = train.sample(frac=1)
k_folder = np.array_split(shuffled, 7)

for df in range(0, 7):
    train_tmp = k_folder[df]
    X = train_tmp.iloc[:, :-1]
    y = train_tmp.iloc[:, data.shape[1]-1]

    #dataset for mutliclass pred
    y_pred_df = pd.DataFrame(columns = range(1, 8)) 
    
    for number_class in range(1, 8):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1234)

        #change values for different decision stumps
        y_train[y_train != number_class] = -1
        y_train[y_train == number_class] = 1

        y_test[y_test != number_class] = -1
        y_test[y_test == number_class] = 1


        # Adaboost classification with 100 weak classifiers
        clf = Adaboost(n_clf=T_round[df])
        clf.fit(X_train, y_train)
        y_pred, y_pred_value = clf.predict(X_test)

        y_pred_df[number_class] = y_pred_value

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1234) 
    final_pred = y_pred_df.idxmax(axis = 1)
    print('With T = ', T_round[df])
    print("Accuracy weighted:", accuracy_score(y_test.reset_index(drop=True), final_pred.reset_index(drop=True)))
    print("F1 weighted:", f1_score(y_test.reset_index(drop=True), final_pred.reset_index(drop=True), average = 'weighted'))
    print("Precision weighted:", precision_score(y_test.reset_index(drop=True), final_pred.reset_index(drop=True), average = 'weighted'))
    print("Recall weighted:", recall_score(y_test.reset_index(drop=True), final_pred.reset_index(drop=True), average = 'weighted'))
    print('Matthews Corrcoef', matthews_corrcoef(y_test.reset_index(drop=True), final_pred.reset_index(drop=True)))
    print('Confusion Matrix\n', confusion_matrix(y_test.reset_index(drop=True), final_pred.reset_index(drop=True)))
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))    

In [ ]:
# EXTERNAL CROSS VALIDATION (hyperparamethers already defined: T = 100, learning_rate = 0.6)
start_time = datetime.now()
number_folder = 5 # K folder
#import dataset
#data = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\covtype.csv", header = 0, sep=",")
data = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct1.csv")
data2 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct2.csv")
data3 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct3.csv")
data4 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct4.csv")
data5 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct5.csv")
data6 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct6.csv")
data7 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct7.csv")
data = data.append(data2)
data = data.append(data3)
data = data.append(data4)
data = data.append(data5)
data = data.append(data6)
data = data.append(data7)
data_cont = data.loc[:, 'Elevation':'Horizontal_Distance_To_Fire_Points'] # continuous variable
data = data.reset_index(drop = True)
#bunching cont variable
for cont_col in data_cont.columns:
        #check every feat is >0
        if data[cont_col].min() > 0:
            #log transformation
            data[cont_col] = np.log(data[cont_col])
        data[cont_col] = pd.qcut(data[cont_col], q = 15, labels = False) #from continuous to class, range(0, 10)
        
#define X and Y
X_init = data.iloc[:, :-1]
y_init = data.iloc[:, data.shape[1]-1]



#split in train and test (for each binary classifier we have the same train and test)
X_train_init, X_test_init, y_train_init, y_test_init = train_test_split(X_init, y_init, test_size = 0.20, random_state = 1234)

#shuffle initial dataset (without following the class' order)
data = data.reset_index(drop = True)
data_init_shuffle = data.sample(frac=1)
#different subset D_i
S_index = data.index

for df in range(0, number_folder):
    
    y_pred_df = pd.DataFrame(columns = range(1, 8)) 
    
    for number_class in range(1, 8):
        k_folder = np.array_split(shuffled, number_folder) #same seed (random state) --> same shuffle
        #inside first df
        D_i = k_folder[df]
        #get D_i indeces
        D_i_index = D_i.index
        #get S_i indeces as differences between data indeced and D_i indeces
        index_S_i = S_index.difference(D_i_index)

        #def S_i
        S_i = data.loc[index_S_i, :]
        
        #X e y D_i
        X_D_i = D_i.iloc[:, :-1]
        y_D_i = D_i.iloc[:, data.shape[1]-1]
                
        #X e y S_i
        X_S_i = S_i.iloc[:, :-1]
        y_S_i = S_i.iloc[:, data.shape[1]-1]
        
        #change values for different decision stumps
        y_S_i[y_S_i != number_class] = -1
        y_S_i[y_S_i == number_class] = 1

        y_D_i[y_D_i != number_class] = -1
        y_D_i[y_D_i == number_class] = 1
        
        # Adaboost classification with 100 weak classifiers
        clf = Adaboost(n_clf=100)
        clf.fit(X_S_i, y_S_i)
        y_pred, y_pred_value = clf.predict(X_D_i)

        y_pred_df[number_class] = y_pred_value
    
    k_folder = np.array_split(shuffled, number_folder)
    D_i = k_folder[df]
    y_D_i = D_i.iloc[:, data.shape[1]-1]
    final_pred = y_pred_df.idxmax(axis = 1)
    print('Folder = ', df+1)
    print("Accuracy weighted:", accuracy_score(y_D_i.reset_index(drop=True), final_pred.reset_index(drop=True)))
    print("F1 weighted:", f1_score(y_D_i.reset_index(drop=True), final_pred.reset_index(drop=True), average = 'weighted'))
    print("Precision weighted:", precision_score(y_D_i.reset_index(drop=True), final_pred.reset_index(drop=True), average = 'weighted'))
    print("Recall weighted:", recall_score(y_D_i.reset_index(drop=True), final_pred.reset_index(drop=True), average = 'weighted'))
    print('Matthews Corrcoef', matthews_corrcoef(y_D_i.reset_index(drop=True), final_pred.reset_index(drop=True)))
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))    

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
#import dataset
#data = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\covtype.csv", header = 0, sep=",")
data = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct1.csv")
data2 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct2.csv")
data3 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct3.csv")
data4 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct4.csv")
data5 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct5.csv")
data6 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct6.csv")
data7 = pd.read_csv(r"C:\Users\Luca\Desktop\UNI\STATALE\LEZIONI\ML\PROJECT\ct7.csv")
data = data.append(data2)
data = data.append(data3)
data = data.append(data4)
data = data.append(data5)
data = data.append(data6)
data = data.append(data7)
data_cont = data.loc[:, 'Elevation':'Horizontal_Distance_To_Fire_Points'] # continuous variable
data = data.reset_index(drop = True)
#bunching cont variable
for cont_col in data_cont.columns:
        #check every feat is >0
        if data[cont_col].min() > 0:
            #log transformation
            data[cont_col] = np.log(data[cont_col])
        data[cont_col] = pd.qcut(data[cont_col], q = 15, labels = False) #from continuous to class, range(0, 10)
        
#define X and Y
X_init = data.iloc[:, :-1]
y_init = data.iloc[:, data.shape[1]-1]



#split in train and test (for each binary classifier we have the same train and test)
X_train_init, X_test_init, y_train_init, y_test_init = train_test_split(X_init, y_init, test_size = 0.20, random_state = 1234) 


regr_2 = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate = 0.6)

regr_2.fit(X_train_init, y_train_init)
pred = regr_2.predict(X_test_init)

print('With T = 100' )
print("Accuracy weighted:", accuracy_score(y_test_init, pred))
print("F1 weighted:", f1_score(y_test_init, pred, average = 'weighted'))
print("Precision weighted:", precision_score(y_test_init, pred, average = 'weighted'))
print("Recall weighted:", recall_score(y_test_init, pred, average = 'weighted'))
print('Matthews Corrcoef', matthews_corrcoef(y_test_init, pred))
print('Confusion Matrix\n', confusion_matrix(y_test_init, pred))